# Ejercicio 1 - Crear un kernel a partir de una función

Partiendo del código suministrado, modificarlo para realizar el cálculo en la GPU.

---

In [9]:
# Ejecutar en Google Colab
!pip install numpy matplotlib scikit-image numba cython setuptools

In [10]:
### EVITAR ERRORES

!uv pip install -q --system numba-cuda==0.4.0

from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

In [24]:
import numpy as np
from numba import cuda

n = 16384 # nº etos

# En CPU
def h_square(a):
    return a**2

# En GPU
@cuda.jit # para aprovechar el paralelismo
def d_square(d_a, d_out):
    idx = cuda.grid(1) # coger índice de cada hilo (1 dimensión)
    d_out[idx] = d_a[idx]**2 # poner en cuadrado el elemento del array

# Array CPU
a = np.arange(n, dtype=np.float32)

# Vbles GPU
d_a = cuda.to_device(a) # copia `a` a gpu
d_out = cuda.device_array_like(d_a) # guarda espaico en memoria para la solución

# Bloques e hilos (blocks*threads=n)
threads = 256 # MÚLTIPLO DE 32!!! (warps)
blocks = int(n//threads) # para que el total de n (blocks*threads=n)

# Lanzar kernel
d_square[blocks, threads](d_a, d_out)

# cuda synchronized para esperar a que terminen todos los hilos
cuda.synchronize() # como una barrera para sincronizar y que terminen todos

# Resultado CPU
out = h_square(a)
aux_out = d_out.copy_to_host().astype(np.float32)

# Assert
np.testing.assert_almost_equal(aux_out, out)

# Medición tiempo

# cpu
print("\nTiempo en CPU:")
%timeit h_square(a)

# gpu
print("\nTiempo en GPU:")
%timeit d_square[blocks, threads](d_a, d_out); cuda.synchronize()

/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/dispatcher.py:605: NumbaPerformanceWarning: Grid size 64 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))



Tiempo en CPU:
3.83 µs ± 867 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Tiempo en GPU:
51.1 µs ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
